# Data Science For Biology 2025 Final Project: Mutagenesis Development and Lead Protein Sequence Screening

## Libraries and Packages

In [1]:
import pandas as pd
import numpy as np
from collections import Counter
import re
import joblib
from tqdm import tqdm

# Set random seed for reproducibility
np.random.seed(42)

## Identify AD Sequences With Low SCGlucose Activity

In [2]:
data = pd.read_csv("./data/gcn4-orthologs.csv")
data = data.rename(columns={data.columns[0]: "tile_id"})
data.head()

,tile_id,DNAseq,ADseq,numreads_per_AD_7AGFP_AAS,Activity_7AGFP_AAS,numreads_per_AD_8AmCh_AAS,Activity_8AmCh_AAS,numreads_per_AD_9ARatio_AAS,Activity_9ARatio_AAS,numreads_per_AD_10BRatio_AAS,...,Activity_BYS4_BioRepAverage,Stdev_BYS4_BioReps,Activity_SCglucose,Start,Recovered,Activity_SCgalactose,Activity_P3promoter,Activity_P14promoter,Activity_P15promoter,Activity_P3promoterMig1TFBSremoved
0,0,AAAAATTCTAGATTTGTTTCTTTGATGCAAGGATTCTCTGATGGTT...,KNSRFVSLMQGFSDGSKDIIDNEVREVGDFSVNKPPQADD,NaN,NaN,NaN,NaN,NaN,NaN,3151.0,...,NaN,NaN,11666.48682,341,True,97250.68362,NaN,11149.49032,NaN,8040.343648
1,1,AAAAATTCTCCATCTGGTTTGGCTCATTCTTCTGTTTCTGGTGTTG...,KNSPSGLAHSSVSGVGKSRRRKGPLPPITVQDPSDKTAIK,942.0,709.606092,367.0,2921.917252,601.0,22242.74698,6909.0,...,21968.05843,5849.143385,21968.05843,146,True,15516.96245,14570.13217,16872.28649,11555.08411,10255.296840
2,2,AAACAAAATTCTCAAGTTGATCAATCTCCATTGTTGCCAGAAGAAG...,KQNSQVDQSPLLPEEELEVSEHLRRRRSGTATSPPAGGVR,NaN,NaN,NaN,NaN,NaN,NaN,6577.0,...,NaN,NaN,21203.02478,326,True,0.00000,18274.98241,15702.60901,18303.29201,17265.783330
3,3,AAACAAAGAGCTACTCCATTGACTCCAGTTGTTCCAGAATCTGATG...,KQRATPLTPVVPESDDPVALKRARNTEAARRSRARKLERM,357.0,1039.469842,342.0,1089.428196,450.0,74251.36241,NaN,...,NaN,NaN,47244.41248,246,True,92844.00000,NaN,NaN,NaN,84807.518160
4,4,AAACAAAGATCTATTCCATTGTCTCCAATTGTTCCAGAATCTTCTG...,KQRSIPLSPIVPESSDPAALKRARNTEAARRSRARKLQRM,539.0,1081.980998,627.0,622.131501,586.0,58872.95132,128.0,...,95093.69451,13292.686710,95093.69451,211,True,76399.62949,NaN,125056.17980,129302.35080,100360.735500


In [3]:
# Sort by activity_SCglucose
activity = "Activity_SCglucose"
low_activity_df = data.sort_values(by=[activity], ascending=True)
low_activity_df = low_activity_df[['ADseq', activity]].dropna()
low_activity_df


,ADseq,Activity_SCglucose
9175,DFVLFDSPQPQRTTVNRPSSVPSNSAAPFGSLQSNTTSTN,6138.0
9585,DDKAAGDSAPVDSAPALVKQPSTTPLDSPAPLIMDTYTRR,6138.0
3711,RKLERQDVMERRIAELEKSLEEAEQREQYWKAMAQAQTQV,6138.0
2712,TLAARKSRQRKMQRFEELEDQIAKLEAERDHWKEIALRRS,6138.0
8564,EAARRSRARKMERMNQLEDKVEDLVGEKQALQDEVDRLKS,6138.0
...,...,...
6161,HANRGPDFDALFDLTANSFVDGLDAASLAMFDTQQLDKVQ,262143.0
1541,TTFNNTQAQQEFPSIMSTSSIPQDFDALFDLTANSFVDGL,262143.0
11857,GTISPQDLMMDASAPPSASLTDLSTPSFESPGYFSQDPSP,262143.0
6726,PMFEFESLDESNDPKNWTSLFENDLPIITEDDVSLNDKAI,262143.0


In [4]:
# Let's look at the min and max values of the activity
low_activity_df.describe()

,Activity_SCglucose
count,18944.000000
mean,66087.894674
std,61869.279225
min,6138.000000
25%,24438.359877
50%,43490.494810
75%,77276.723883
max,262143.000000


## Pointwise Mutatagenesis
Without loss of generality, let's simply pick the protein sequence with the lowest activity and do some random pointwise mutations from a given list of amino acids

In [5]:
# FEATURES

# variables I need glovally
AA_LIST = 'ACDEFGHIKLMNPQRSTVWY'
AA_COLS = [f"AA_{aa}" for aa in AA_LIST]

# I use Kyte-Doolittle scale for hydrophobicity hereee CHECK WITH PROF STALLER
# https://www.rbvi.ucsf.edu/chimera/docs/UsersGuide/midas/hydrophob.html
KD_HYDROPHOBICITY = {
    'A': 1.8, 'C': 2.5, 'D': -3.5, 'E': -3.5, 'F': 2.8,
    'G': -0.4, 'H': -3.2, 'I': 4.5, 'K': -3.9, 'L': 3.8,
    'M': 1.9, 'N': -3.5, 'P': -1.6, 'Q': -3.5, 'R': -4.5,
    'S': -0.8, 'T': -0.7, 'V': 4.2, 'W': -0.9, 'Y': -1.3
}

# i use the tuple here for naming the motifs
MOTIFS = [
    ("W..LF", "W..LF"),
    ("WFYL_WFYL", "[WFYL]..[WFYL][WFYL]"),
    ("WFL_WFL", "[WFL]..[WFL][WFL]"),
    ("DE_WFY", "[DE][WFY]"),
    ("DE_WF", "[DE][WF]"),
    ("DE_L", "[DE][L]"),
    ("DE_x_WFY", "[DE].[WFY]"),
    ("DE_xx_WFY", "[DE]..[WFY]"),
    ("FF", "FF"),
    ("F_F", "F.F"),
    ("F__F", "F..F"),
    ("FY_5x_FY", "[FY].....[FY]"),
    ("SP", "SP"),
    ("WFYL_3x_WFYL", "[WFYL]...[WFYL][WFYL]")
]

# FUNCTIONS TO CALC

def amino_acid_count(sequence: str) -> dict[str, int]:
    c = Counter(sequence)
    return {f"AA_{aa}": c.get(aa, 0) for aa in AA_LIST}

def net_charge(seq: str) -> int:
    charge_map = {"K": 1, "R": 1, "D": -1, "E": -1}
    return sum(charge_map.get(a, 0) for a in seq)

def hydrophobicity(sequence: str) -> float:
    total_hydrophobicity = sum(KD_HYDROPHOBICITY[aa] for aa in sequence if aa in KD_HYDROPHOBICITY)
    return total_hydrophobicity

def motif_counts_dict(seq: str) -> dict[str, int]:
    return {name: len(re.findall(pat, seq)) for name, pat in MOTIFS}

# APPLY DA FEATURESSSS
def add_sequence_features(df: pd.DataFrame, seq_col: str = "ADseq") -> pd.DataFrame:
    # AA composition
    aa_expanded = df[seq_col].apply(amino_acid_count).apply(pd.Series)
    
    # Motif counts
    motif_expanded = df[seq_col].apply(motif_counts_dict).apply(pd.Series)

    # Scalar features
    df["NetCharge"] = df[seq_col].apply(net_charge)
    df["Hydrophobicity"] = df[seq_col].apply(hydrophobicity)
    
    # Sequence length
    df["Length"] = df[seq_col].str.len()

    # Combine everything
    result_df = pd.concat([df, aa_expanded, motif_expanded], axis=1)

    return result_df

In [6]:
low_activity_df = add_sequence_features(low_activity_df, seq_col="ADseq")

# now we put Activity_SCglucose as the last column for convenience
low_activity_df = low_activity_df[[col for col in low_activity_df.columns if col != activity] + [activity]]
low_activity_df.head()

,ADseq,NetCharge,Hydrophobicity,Length,AA_A,AA_C,AA_D,AA_E,AA_F,AA_G,...,DE_L,DE_x_WFY,DE_xx_WFY,FF,F_F,F__F,FY_5x_FY,SP,WFYL_3x_WFYL,Activity_SCglucose
9175,DFVLFDSPQPQRTTVNRPSSVPSNSAAPFGSLQSNTTSTN,0,-26.6,40,2,0,2,0,3,1,...,0,0,0,0,0,1,0,1,0,6138.0
9585,DDKAAGDSAPVDSAPALVKQPSTTPLDSPAPLIMDTYTRR,-2,-21.6,40,6,0,6,0,0,1,...,0,1,0,0,0,0,0,1,0,6138.0
3711,RKLERQDVMERRIAELEKSLEEAEQREQYWKAMAQAQTQV,-1,-53.3,40,5,0,1,8,0,0,...,1,1,1,0,0,0,0,0,0,6138.0
2712,TLAARKSRQRKMQRFEELEDQIAKLEAERDHWKEIALRRS,3,-54.1,40,5,0,2,6,1,0,...,1,1,0,0,0,0,0,0,0,6138.0
8564,EAARRSRARKMERMNQLEDKVEDLVGEKQALQDEVDRLKS,0,-54.8,40,4,0,4,6,0,1,...,1,0,0,0,0,0,0,0,0,6138.0


In [7]:
protein_sequence = low_activity_df.iloc[0, 0]
activity_value = low_activity_df.iloc[0, -1]
print(f"Protein sequence: {protein_sequence}")
print(f"Activity value: {activity_value}")

Protein sequence: DFVLFDSPQPQRTTVNRPSSVPSNSAAPFGSLQSNTTSTN
Activity value: 6138.0


In [8]:
# mutate the protein sequence pointwise
def mutate_sequence(seq: str, pos: int, new_aa: str) -> str:
    if pos < 0 or pos >= len(seq):
        raise ValueError("Position out of range")
    if new_aa not in AA_LIST:
        raise ValueError("Invalid amino acid")
    return seq[:pos] + new_aa + seq[pos + 1:]

def simulate_mutations_over_generations(
    initial_sequence,
    aa_list,
    mutate_fn,
    add_features_fn,
    num_generations=10,
    num_mutations_per_generation=1000,
    max_sequences_per_gen=5000  # optional cap
):
    """Simulate mutations across multiple generations (optimized)."""
    current_sequences = [initial_sequence]
    all_generations = []

    for gen in tqdm(range(num_generations), desc="Simulating generations", position=0, leave=True):
        mutated_sequences = set()

        for seq in tqdm(current_sequences, desc=f"Mutating sequences (Gen {gen+1})", position=1, leave=False):
            seq_array = np.array(list(seq))
            mutation_positions = np.random.randint(0, len(seq), size=num_mutations_per_generation)
            mutation_aas = np.random.choice(aa_list, size=num_mutations_per_generation)

            for pos, new_aa in zip(mutation_positions, mutation_aas):
                new_seq = seq_array.copy()
                new_seq[pos] = new_aa
                mutated_sequences.add("".join(new_seq))
        
        # Optionally limit number of sequences
        if len(mutated_sequences) > max_sequences_per_gen:
            mutated_sequences = np.random.choice(
                list(mutated_sequences),
                size=max_sequences_per_gen,
                replace=False
            ).tolist()
        else:
            mutated_sequences = list(mutated_sequences)

        mutated_df = pd.DataFrame(mutated_sequences, columns=["ADseq"])
        mutated_df["Generation"] = gen + 1

        mutated_df = add_features_fn(mutated_df, seq_col="ADseq")
        all_generations.append(mutated_df)

        current_sequences = mutated_sequences

    result_df = pd.concat(all_generations, ignore_index=True)
    return result_df


In [9]:
result = simulate_mutations_over_generations(
    protein_sequence, 
    list(AA_LIST), 
    mutate_sequence, 
    add_sequence_features,
    num_generations=5, 
    num_mutations_per_generation=1000
)

result.head()


Simulating generations: 100%|██████████| 5/5 [04:36<00:00, 55.28s/it]


,ADseq,Generation,NetCharge,Hydrophobicity,Length,AA_A,AA_C,AA_D,AA_E,AA_F,...,DE_WF,DE_L,DE_x_WFY,DE_xx_WFY,FF,F_F,F__F,FY_5x_FY,SP,WFYL_3x_WFYL
0,DFVLFDSPQPQRTTVNRPSSVPSNSAAPFGSLFSNTTSTN,1,0,-20.3,40,2,0,2,0,4,...,1,0,0,0,0,0,1,0,1,0
1,DFVLFDSPQPQRTTVNRPSSVPSNSAAPFGSLQSNTYSTN,1,0,-27.2,40,2,0,2,0,3,...,1,0,0,0,0,0,1,0,1,0
2,DFVLFDSPQPQRTTVNRPSNVPSNSAAPFGSLQSNTTSTN,1,0,-29.3,40,2,0,2,0,3,...,1,0,0,0,0,0,1,0,1,0
3,DFVCFDSPQPQRTTVNRPSSVPSNSAAPFGSLQSNTTSTN,1,0,-27.9,40,2,1,2,0,3,...,1,0,0,0,0,0,1,0,1,0
4,DFVLFDSPQPQRTTVNRPSSVPSNSAAPFGSLQSNTKSTN,1,1,-29.8,40,2,0,2,0,3,...,1,0,0,0,0,0,1,0,1,0


In [10]:
# Load the regression model
model = joblib.load("./models/random_forest_model.pkl")

mutated_feature = result.drop(columns=["ADseq", "Generation"])

# Make predictions on the mutated sequence
predictions = model.predict(mutated_feature)

# Add predictions to the DataFrame
result["Activity_SCglucose"] = predictions

result.head()

,ADseq,Generation,NetCharge,Hydrophobicity,Length,AA_A,AA_C,AA_D,AA_E,AA_F,...,DE_L,DE_x_WFY,DE_xx_WFY,FF,F_F,F__F,FY_5x_FY,SP,WFYL_3x_WFYL,Activity_SCglucose
0,DFVLFDSPQPQRTTVNRPSSVPSNSAAPFGSLFSNTTSTN,1,0,-20.3,40,2,0,2,0,4,...,0,0,0,0,0,1,0,1,0,53178.690049
1,DFVLFDSPQPQRTTVNRPSSVPSNSAAPFGSLQSNTYSTN,1,0,-27.2,40,2,0,2,0,3,...,0,0,0,0,0,1,0,1,0,39020.518628
2,DFVLFDSPQPQRTTVNRPSNVPSNSAAPFGSLQSNTTSTN,1,0,-29.3,40,2,0,2,0,3,...,0,0,0,0,0,1,0,1,0,33516.371032
3,DFVCFDSPQPQRTTVNRPSSVPSNSAAPFGSLQSNTTSTN,1,0,-27.9,40,2,1,2,0,3,...,0,0,0,0,0,1,0,1,0,40665.899865
4,DFVLFDSPQPQRTTVNRPSSVPSNSAAPFGSLQSNTKSTN,1,1,-29.8,40,2,0,2,0,3,...,0,0,0,0,0,1,0,1,0,29343.319507


In [11]:
highest_activity_candidates = result.sort_values(by=["Activity_SCglucose"], ascending=False)
highest_activity_candidates

,ADseq,Generation,NetCharge,Hydrophobicity,Length,AA_A,AA_C,AA_D,AA_E,AA_F,...,DE_L,DE_x_WFY,DE_xx_WFY,FF,F_F,F__F,FY_5x_FY,SP,WFYL_3x_WFYL,Activity_SCglucose
16239,DFDLFDSPQPQETTVNRISEVPSNSAAPFGSLFSNTTSTN,5,-4,-23.6,40,2,0,3,2,4,...,1,1,0,0,0,1,0,1,0,127012.216605
15924,DFVLFDSWQPQHTTVNCPSSVPSNSAEPFGSLQSNTFSTN,5,-3,-19.4,40,1,1,2,1,4,...,0,2,0,0,0,1,0,0,0,111171.151688
13287,DFVYFDSFWPQRTTVNRPSSVPSNSAAKFGSLQSNTTSTN,4,1,-27.0,40,2,0,2,0,4,...,0,1,2,0,0,1,1,0,1,105984.463516
19199,DFVLFDSPQPQLCYVDCPSSVPSNSAAPFGSLQSNTTSTN,5,-3,-8.7,40,2,2,3,0,3,...,0,0,0,0,0,1,0,1,0,94451.797448
18400,DFVLFDSPQPQATTVNDPSSVLSCSAAPFGSLQSNTTSTC,5,-3,-1.9,40,3,2,3,0,3,...,0,0,0,0,0,1,0,1,0,94073.384835
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19292,DQVLSDSPQPQRTTVNSPSSMPSNSAAPFGSLQSNTTSTG,5,-1,-32.0,40,2,0,2,0,1,...,0,0,0,0,0,0,0,2,0,21210.357949
17346,DKVLFDSPGPQRTTVNRPSSKPSNSAAPAGSLQSNTCSTN,5,2,-36.1,40,3,1,2,0,1,...,0,0,0,0,0,0,0,1,0,20955.824968
13381,DMVLFDSPQPQRTTTNRPSSVPSNSAAPGDSLQSNTTSTN,4,-1,-38.7,40,2,0,3,0,1,...,0,0,0,0,0,0,0,1,0,20925.780502
20060,DCVLFDSPQPQRTTVNRPSSKPSNSAAPAGSLQSNTNSTN,5,1,-38.8,40,3,1,2,0,1,...,0,0,0,0,0,0,0,1,0,20886.300750


### Looking Into The Difference

In [13]:
top_candidate = highest_activity_candidates.iloc[0]
print(f"Top candidate sequence: {top_candidate['ADseq']}")
print(f"Original sequence: {protein_sequence}")
print(f"Original activity: {activity_value}")
print(f"Predicted activity: {top_candidate['Activity_SCglucose']}")

Top candidate sequence: DFDLFDSPQPQETTVNRISEVPSNSAAPFGSLFSNTTSTN
Original sequence: DFVLFDSPQPQRTTVNRPSSVPSNSAAPFGSLQSNTTSTN
Original activity: 6138.0
Predicted activity: 127012.2166045


In [ ]:
# i wanna see how these two are different
def compare_sequences(seq1: str, seq2: str) -> list[tuple[int, str, str]]:
    differences = []
    for i, (a, b) in enumerate(zip(seq1, seq2)):
        if a != b:
            differences.append((i, a, b))
    return differences
differences = compare_sequences(protein_sequence, top_candidate["ADseq"])
print("Differences between original and top candidate:")
for pos, original_aa, new_aa in differences:
    print(f"Position {pos}: {original_aa} -> {new_aa}")

Differences between original and top candidate:
Position 2: V -> D
Position 11: R -> E
Position 17: P -> I
Position 19: S -> E
Position 32: Q -> F


In [18]:
# Let's check the difference in net charge and hydrophobicity
original_net_charge = net_charge(protein_sequence)
top_candidate_net_charge = net_charge(top_candidate["ADseq"])
print(f"Original net charge: {original_net_charge}")
print(f"Top candidate net charge: {top_candidate_net_charge}")
print(f"Net charge difference: {top_candidate_net_charge - original_net_charge}")


original_hydrophobicity = hydrophobicity(protein_sequence)
top_candidate_hydrophobicity = hydrophobicity(top_candidate["ADseq"])
print(f"Original hydrophobicity: {original_hydrophobicity}")
print(f"Top candidate hydrophobicity: {top_candidate_hydrophobicity}")
print(f"Hydrophobicity difference: {top_candidate_hydrophobicity - original_hydrophobicity}")

Original net charge: 0
Top candidate net charge: -4
Net charge difference: -4
Original hydrophobicity: -26.599999999999998
Top candidate hydrophobicity: -23.6
Hydrophobicity difference: 2.9999999999999964
